<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Modulo_4/Projeto_Final_Tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [7]:
movies = pd.read_csv("https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/movies.csv?raw=true")
series = pd.read_csv("https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/series.csv?raw=true")

In [5]:
movies.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,Casting,Crew,budget,genres,runtime,status
0,False,/fQq1FWp1rC89xDrRMuyFJdFUdMd.jpg,"[10749, 35]",761053,en,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,33.315,/fYtHxTxlhzD4QWfEbrC1rypysSD.jpg,2020-11-19,Gabriel's Inferno Part III,False,8.8,725,Giulio Berruti;Rhett Wellington;Melanie Zanett...,Tosca Musk,0,Romance;Comedy,105.0,Released
1,False,/jtAI6OJIWLWiRItNSZoWjrsUtmi.jpg,[10749],724089,en,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,9.365,/pci1ArYW7oJ2eyTo2NMYEKHHiCP.jpg,2020-07-31,Gabriel's Inferno Part II,False,8.8,1179,Melanie Zanetti;Giulio Berruti;James Andrew Fr...,Tosca Musk,0,Romance,105.0,Released
2,False,/xRI636TOdS1K1GBqIBRSmfZ1T5x.jpg,"[35, 18, 10749]",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",15.250,/2CAL2433ZeIihfX1Hb2139CX0pW.jpg,1995-10-20,Dilwale Dulhania Le Jayenge,False,8.8,2663,Shah Rukh Khan;Kajol;Amrish Puri;Anupam Kher;S...,Aditya Chopra,13200000,Comedy;Drama;Romance,190.0,Released
3,False,/mxn0RJ6stopje13MZdwU35XDnAx.jpg,"[10751, 12, 16, 14]",441130,en,Wolfwalkers,"In a time of superstition and magic, when wolv...",28.370,/ehAKuE48okTuonq6TpsNQj8vFTC.jpg,2020-10-26,Wolfwalkers,False,8.7,258,Honor Kneafsey;Eva Whittaker;Sean Bean;Simon M...,Tomm Moore;Ross Stewart;Mark Mullery,0,Family;Adventure;Animation;Fantasy,103.0,Released
4,False,/tXHpvlr5F7gV5DwgS7M5HBrUi2C.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,54.041,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,1994-09-23,The Shawshank Redemption,False,8.7,18389,Tim Robbins;Morgan Freeman;Bob Gunton;William ...,Frank Darabont;Sioux Richards;Jesse V. Johnson...,25000000,Drama;Crime,142.0,Released


In [8]:
series.head()

,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count,Casting,Crew,genres,last_air_date,number_of_episodes,number_of_seasons,status
0,NaN,2004-05-10,"[16, 35]",100,I Am Not an Animal,['GB'],en,I Am Not an Animal,I Am Not An Animal is an animated comedy serie...,12.076,/qG59J1Q7rpBc1dvku4azbzcqo8h.jpg,9.4,598,Julia Davis;Kevin Eldon;Simon Pegg;Steve Cooga...,NaN,Animation;Comedy,2004-06-14,6,1,Ended
1,/7gbmM2NWcqZONbp65HUWDf4wr0Q.jpg,2019-07-12,"[16, 18]",88040,Given,['JP'],ja,ギヴン,"Tightly clutching his Gibson guitar, Mafuyu Sa...",28.498,/pdDCcAq8RNSZNk81PXYoHNUPHjn.jpg,9.2,391,Fumiya Imai;Shougo Yano;Takuya Eguchi;Masatomo...,Hikaru Yamaguchi,Animation;Drama,2019-09-20,11,1,Ended
2,/uAjMQlbPkVHmUahhCouANlHSDW2.jpg,2019-01-11,"[16, 9648, 10765, 10759, 18]",83097,The Promised Neverland,['JP'],ja,約束のネバーランド,"Surrounded by a forest and a gated entrance, t...",65.812,/yxdeII5tI8qqiERcMxjW9DfB6Gz.jpg,9.2,545,Sumire Morohoshi;Mariya Ise;Lynn;Shinei Ueki;E...,NaN,Animation;Mystery;Sci-Fi & Fantasy;Action & Ad...,2021-02-26,23,2,Returning Series
3,/rnf2BDKeF1sxPEPhAcdvL0auuxr.jpg,2020-10-03,"[16, 35]",100049,TONIKAWA: Over the Moon for You,['JP'],ja,トニカクカワイイ,"First comes marriage, then comes an earthbound...",32.649,/n1oOuXlcHF1brgcgyiwmnIODXso.jpg,9.1,176,Junya Enoki;Akari Kitō;Yū Serizawa;Sumire Uesa...,Hiroshi Ikehata,Animation;Comedy,2020-12-19,12,1,Ended
4,/qSgBzXdu6QwVVeqOYOlHolkLRxZ.jpg,2019-01-09,"[16, 10759, 10765, 18]",83095,The Rising of the Shield Hero,['JP'],ja,盾の勇者の成り上がり,Iwatani Naofumi was summoned into a parallel w...,15.476,/6cXf5EDwVhsRv8GlBzUTVnWuk8Z.jpg,9.1,457,Kaito Ishikawa;Asami Seto;Rina Hidaka;Hiroki Y...,Takao Abo,Animation;Action & Adventure;Sci-Fi & Fantasy;...,2019-06-26,25,2,Returning Series
